- MODELO REGRESIÓN LINEAL:

Target = "alcohol_%" 

División en datos de entrenamiento y datos de prueba:

In [ ]:
X_reg = beer_ML.loc[:, ~beer_ML.columns.isin(["nombre", "estilo", "alcohol_%","opinion_aroma", 
                                              "opinion_apariencia", "opinion_paladar", 
                                              "opinion_sabor", "opinion_general"])] 
y_reg = beer_ML.loc[:, 'alcohol_%']

X_train, X_test, y_train, y_test = train_test_split(
                                        X_reg,
                                        y_reg,
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True)

Escalado:

In [ ]:
birra_escalado_reg = StandardScaler()

x_train_esc = birra_escalado_reg.fit_transform(X_train)
x_test_esc = birra_escalado_reg.transform(X_test)

birra_escalado_reg

StandardScaler()

Ridge / Lasso / Cross-Validation:

In [ ]:
ridge_model = Ridge(alpha=0.5)
lasso_model = Lasso(alpha=0.5)

ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

ridge_score = ridge_model.score(X_test, y_test)
lasso_score = lasso_model.score(X_test, y_test)

print(f"Ridge R2 Score: {ridge_score}")
print(f"Lasso R2 Score: {lasso_score}")

Ridge R2 Score: 0.4067364700856133
Lasso R2 Score: 0.4093204472977755


In [ ]:
model = LinearRegression()

scores = cross_val_score(model, x_train_esc, y_train, cv=10, scoring='neg_mean_squared_error')

mean_mse = -scores.mean()
print(f"Error cuadrático medio promedio: {mean_mse:.2f}")

Error cuadrático medio promedio: 1.80


Entrenamiento:

In [ ]:
modelo_lineal = LinearRegression()

modelo_lineal.fit(x_train_esc, y_train)
y_pred = modelo_lineal.predict(x_test_esc)
modelo_lineal

LinearRegression()

Evaluación:

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"mse: {mse:.4f}")
print(f"r2: {r2:.4f}")
print(f"mae: {mae:.4f}")
print(f"rmse: {rmse:.4f}")

mse: 1.9249
r2: 0.4067
mae: 0.9879
rmse: 1.3874


- MODELO REGRESIÓN POLINÓMICA

In [ ]:
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(X_train)

model_poly = LinearRegression()
model_poly.fit(X_poly, y_train)

X_test_poly = poly.transform(X_test)
y_pred_poly = model_poly.predict(X_test_poly)

mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
mae_poly = mean_absolute_error(y_test, y_pred_poly)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))

print(f"mse: {mse_poly:.4f}")
print(f"r2: {r2_poly:.4f}")
print(f"mae: {mae_poly:.4f}")
print(f"rmse: {rmse_poly:.4f}")

mse: 2.6318
r2: 0.1889
mae: 1.1129
rmse: 1.6223


- MODELO SUPPORT VECTOR MACHINE

División en datos de entrenamiento y datos de prueba:

In [ ]:
X_reg_svm = beer_ML.loc[:, ~beer_ML.columns.isin(["nombre", "estilo", "alcohol_%","opinion_aroma", 
                                              "opinion_apariencia", "opinion_paladar", 
                                              "opinion_sabor", "opinion_general"])] 
y_reg_svm = beer_ML.loc[:, 'alcohol_%']

x_train, x_test, y_train, y_test = train_test_split(X_reg_svm, y_reg_svm, test_size=0.2, random_state=42)

Escalado:

In [ ]:
birra_escalado_SVM = StandardScaler()

x_train_esc_SVM = birra_escalado_SVM.fit_transform(X_train)
x_test_esc_SVM = birra_escalado_SVM.transform(X_test)

birra_escalado_SVM

StandardScaler()

Cross-Validation:

In [ ]:
modelo_svm = SVR(kernel='linear', C=25, epsilon=15)

In [ ]:
scores = cross_val_score(modelo_svm, x_train_esc_SVM, y_train, cv=5, scoring='neg_mean_squared_error')

mean_mse_svm = -scores.mean()
print(f"Error cuadrático medio promedio: {mean_mse_svm:.2f}")

Error cuadrático medio promedio: 3.20


Entrenamiento:

In [ ]:
modelo_svm.fit(x_train_esc_SVM, y_train)
y_pred_SVM = modelo_svm.predict(X_test)

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Evaluación:

In [ ]:
y_mediana = np.median(y_reg_svm)
y_pred_mediana = np.ones_like(y_reg_svm) * y_mediana

# Calculo el MSE
mse = mean_squared_error(y_test, y_pred)

# Calculo la mediana del MSE
mediana_mse = np.median([mse])

print(mse)
print(f"MSE utilizando la mediana: {mediana_mse:.2f}")

# Comparo el rendimiento del modelo con la mediana
if mse < mediana_mse:
    print("El modelo de regresión lineal mejora respecto a la media.")
else:
    print("El modelo de regresión lineal mejora respecto a la mediana.")

4.69132206227755
MSE utilizando la mediana: 4.69
El modelo de regresión lineal mejora respecto a la mediana.


Grid Search:

In [ ]:
# Definir los valores de hiperparámetros a explorar
parametros = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 1]}

# Crear el modelo SVM para regresión
modelo_svm = SVR(kernel='linear')

# Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(modelo_svm, parametros, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
mejores_parametros = grid_search.best_params_
print(f"Mejores hiperparámetros: {mejores_parametros}")

# Evaluar el modelo con los mejores hiperparámetros en los datos de prueba
y_pred_mejor = grid_search.predict(X_test)
mse_mejor = mean_squared_error(y_test, y_pred_mejor)
print(f"Error cuadrático medio con mejores hiperparámetros: {mse_mejor:.2f}")

Mejores hiperparámetros: {'C': 0.1, 'epsilon': 1}
Error cuadrático medio con mejores hiperparámetros: 3.56


Cross-Validation:

In [ ]:
modelos_reg = {
    "Linear Regression": LinearRegression(),
    "Polinomyal Regression": PolynomialFeatures(),
    "Support Vector Regression": SVR()}

metricas_reg = ["r2", "neg_mean_absolute_error", "neg_mean_squared_error", "neg_root_mean_squared_error"]

resultados_dict_reg = {}

for nombre_modelo, modelo in modelos_reg.items():
    cv_resultados = cross_validate(modelo, X_reg, y_reg, cv=5, scoring=metricas_reg)
    
    for metrica in metricas_reg:
        clave = f"{nombre_modelo}_{metrica}"
        resultados_dict_reg[clave] = cv_resultados[f"test_{metrica}"].mean()

resultados_df_reg = pd.DataFrame([resultados_dict_reg])

c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\feder\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\_re

In [ ]:
resultados_df_reg.T.sort_values(by=0, ascending=False)

,0
Linear Regression_r2,0.358532
Support Vector Regression_r2,0.342049
Linear Regression_neg_mean_absolute_error,-0.999556
Support Vector Regression_neg_mean_absolute_error,-1.001693
Linear Regression_neg_root_mean_squared_error,-1.363799
Support Vector Regression_neg_root_mean_squared_error,-1.384627
Linear Regression_neg_mean_squared_error,-1.877247
Support Vector Regression_neg_mean_squared_error,-1.941986
Polinomyal Regression_r2,NaN
Polinomyal Regression_neg_mean_absolute_error,NaN
